In [6]:
import os
import pandas as pd
from google.cloud import storage

# GCS에서 Parquet 파일 불러오기 및 형식 확인
print("\n--- 2. GCS Parquet 파일 로드 및 형식 확인 ---")
gcs_parquet_path = "gs://sprintda05_final_project/votes/polls_usercandidate.parquet"

print(f"📂 다음 Parquet 파일을 불러옵니다: {gcs_parquet_path}")

# pandas를 사용하여 Parquet 파일 불러오기
# 'engine'을 'pyarrow'로 명시하여 확실하게 pyarrow 사용
# gcsfs가 설치되어 있다면 pandas는 gs:// 경로를 자동으로 처리
df = pd.read_parquet(gcs_parquet_path, engine='pyarrow')

print("\n✅ Parquet 파일 로드 성공!")

print("\n--- 데이터프레임 상위 5개 행 (df.head()) ---")
print(df.head())

print("\n--- 데이터프레임 정보 (df.info()) ---")
# 각 컬럼의 이름, Non-Null 개수, 데이터 타입(Dtype) 확인
df.info()

print("\n--- 데이터프레임 기술 통계 (df.describe()) ---")
# 숫자형 컬럼에 대한 통계 정보 확인
print(df.describe(include='all')) # 모든 컬럼 타입 포함

print(f"\n--- 데이터프레임 행과 열 개수 (df.shape): {df.shape} ---")

print("\n--- 데이터프레임 컬럼 목록 (df.columns) ---")
print(df.columns.tolist())


--- 2. GCS Parquet 파일 로드 및 형식 확인 ---
📂 다음 Parquet 파일을 불러옵니다: gs://sprintda05_final_project/votes/polls_usercandidate.parquet

✅ Parquet 파일 로드 성공!

--- 데이터프레임 상위 5개 행 (df.head()) ---
        id          created_at  question_piece_id  user_id
0  3088872 2023-04-28 12:27:49             998458   849444
1  3088873 2023-04-28 12:27:49             998458   849454
2  3088874 2023-04-28 12:27:49             998458   849460
3  3088875 2023-04-28 12:27:49             998458   849469
4  3088964 2023-04-28 12:28:02             998459   849446

--- 데이터프레임 정보 (df.info()) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4769609 entries, 0 to 4769608
Data columns (total 4 columns):
 #   Column             Dtype         
---  ------             -----         
 0   id                 int64         
 1   created_at         datetime64[ns]
 2   question_piece_id  int64         
 3   user_id            int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 145.6 MB

--- 데이터프레임 기술 통계 (df.descr

In [7]:
df

,id,created_at,question_piece_id,user_id
0,3088872,2023-04-28 12:27:49,998458,849444
1,3088873,2023-04-28 12:27:49,998458,849454
2,3088874,2023-04-28 12:27:49,998458,849460
3,3088875,2023-04-28 12:27:49,998458,849469
4,3088964,2023-04-28 12:28:02,998459,849446
...,...,...,...,...
4769604,646672580,2024-05-08 01:36:00,200139933,857296
4769605,646672581,2024-05-08 01:36:18,200139934,850774
4769606,646672582,2024-05-08 01:36:18,200139934,856446
4769607,646672583,2024-05-08 01:36:18,200139934,857101


In [ ]:
# - **polls_usercandidate**: 질문 조각에 등장하는 유저들 테이블(이준희)
#     - "한 질문마다 최대 8명이 등장합니다”
#         - 8명이 어떻게 선정되는지 알 수 없음
#            작성자가 선택한 친구들인가? 시스템이 자동으로 추천한 유저들인가?
#            서로가 서로의 설문에 있다면 긍정적인 설문일수도 있지 않을까?
#         - 친구를 8명 까지만 받을 수 있나?
#            항상 8명인가?, 8명 보다 적거나 많은 경우는?
#         - 사용자가 질문에 해당하는 8명을 선정한 후 최후의 한 명을 고르는 건가?
#         - 질문의 성격에 따라 자동으로 구분하는지?(예시: 성별로 구분)
#         - 선택지가 8개가 최대숫자인지 아니면 선택지가 무조건 8개인지?

In [25]:
df['created_at'].unique()

<DatetimeArray>
['2023-04-28 12:27:49', '2023-04-28 12:28:02', '2023-04-28 12:28:09',
 '2023-04-28 12:28:16', '2023-04-28 12:28:26', '2023-04-28 12:28:32',
 '2023-04-28 12:28:46', '2023-04-28 12:28:55', '2023-04-28 12:28:57',
 '2023-04-28 12:29:05',
 ...
 '2024-05-07 00:14:26', '2024-05-07 00:14:36', '2024-05-07 00:14:43',
 '2024-05-07 00:14:50', '2024-05-07 00:15:00', '2024-05-07 11:29:07',
 '2024-05-08 01:34:56', '2024-05-08 01:35:02', '2024-05-08 01:36:00',
 '2024-05-08 01:36:18']
Length: 917071, dtype: datetime64[ns]

In [ ]:
# 상호작용이 없는 유저 리스트
non_mutual_users = all_users - mutual_users

# 이 유저들이 등장한 질문 목록
non_mutual_df = df[df['user_id'].isin(non_mutual_users)]

# 각 question에서 등장한 후보 수
candidate_per_question = df.groupby('question_piece_id')['user_id'].nunique()

# 비상상호 유저가 포함된 질문의 후보 수 평균
non_mutual_df['candidate_count'] = non_mutual_df['question_piece_id'].map(candidate_per_question)
print(non_mutual_df['candidate_count'].describe())


count    6905.000000
mean        3.952209
std         0.216708
min         2.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         4.000000
Name: candidate_count, dtype: float64


/tmp/ipykernel_18537/2336628669.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_mutual_df['candidate_count'] = non_mutual_df['question_piece_id'].map(candidate_per_question)


In [ ]:
# 상호작용이 없는 유저 리스트
non_mutual_users = all_users - mutual_users

# 이 유저들이 등장한 질문 목록
non_mutual_df = df[df['user_id'].isin(non_mutual_users)]

# 각 question에서 등장한 후보 수
candidate_per_question = df.groupby('question_piece_id')['user_id'].nunique()

# 비상호 유저가 포함된 질문의 후보 수 평균
non_mutual_df['candidate_count'] = non_mutual_df['question_piece_id'].map(candidate_per_question)
print(non_mutual_df['candidate_count'].describe())


count    6905.000000
mean        3.952209
std         0.216708
min         2.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         4.000000
Name: candidate_count, dtype: float64


/tmp/ipykernel_18537/2336628669.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_mutual_df['candidate_count'] = non_mutual_df['question_piece_id'].map(candidate_per_question)


In [ ]:
# 상호작용이 없는 유저 리스트
non_mutual_users = all_users - mutual_users

# 이 유저들이 등장한 질문 목록
non_mutual_df = df[df['user_id'].isin(non_mutual_users)]

# 각 question에서 등장한 후보 수
candidate_per_question = df.groupby('question_piece_id')['user_id'].nunique()

# 비상호 유저가 포함된 질문의 후보 수 평균
non_mutual_df['candidate_count'] = non_mutual_df['question_piece_id'].map(candidate_per_question)
print(non_mutual_df['candidate_count'].describe())


count    6905.000000
mean        3.952209
std         0.216708
min         2.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         4.000000
Name: candidate_count, dtype: float64


/tmp/ipykernel_18537/2336628669.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_mutual_df['candidate_count'] = non_mutual_df['question_piece_id'].map(candidate_per_question)


In [ ]:
# 상호작용이 없는 유저 리스트
non_mutual_users = all_users - mutual_users

# 이 유저들이 등장한 질문 목록
non_mutual_df = df[df['user_id'].isin(non_mutual_users)]

# 각 question에서 등장한 후보 수
candidate_per_question = df.groupby('question_piece_id')['user_id'].nunique()

# 비상호 유저가 포함된 질문의 후보 수 평균
non_mutual_df['candidate_count'] = non_mutual_df['question_piece_id'].map(candidate_per_question)
print(non_mutual_df['candidate_count'].describe())


count    6905.000000
mean        3.952209
std         0.216708
min         2.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         4.000000
Name: candidate_count, dtype: float64


/tmp/ipykernel_18537/2336628669.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_mutual_df['candidate_count'] = non_mutual_df['question_piece_id'].map(candidate_per_question)


In [9]:
df['question_piece_id'].value_counts()

question_piece_id
9494270      16
14806469     16
28856618     16
124552858    12
80794775     12
             ..
52696110      1
1051419       1
27170355      1
3836336       1
1158226       1
Name: count, Length: 1217589, dtype: int64

In [10]:
df['user_id'].value_counts()

user_id
860304     4226
940572     4070
1137034    3948
1207066    3920
994573     3674
           ... 
1083929       1
988375        1
949239        1
1100661       1
949988        1
Name: count, Length: 19994, dtype: int64

In [11]:
candidate_counts = df.groupby('question_piece_id')['user_id'].count()

In [12]:
max_candidates = candidate_counts.max()
print(f'한 질문에 등장한 최대 후보 수: {max_candidates}명')

한 질문에 등장한 최대 후보 수: 16명


In [13]:
candidate_unique = df.groupby('question_piece_id')['user_id'].nunique()

In [14]:
unique_candidates = candidate_unique.max()
print(f'한 질문에 등장한 유니크 후보 수: {max_candidates}명')

한 질문에 등장한 유니크 후보 수: 16명


In [15]:
user_question_counts = df.groupby('user_id')['question_piece_id'].nunique()


In [16]:
user_question_counts

user_id
833112       5
833113      69
833154      19
833202       2
833203     256
          ... 
1579418      1
1579422      6
1579659      3
1580578      3
1580629      1
Name: question_piece_id, Length: 19994, dtype: int64

In [43]:
# 질문별로 등장하는 유저 ID들을 리스트로 묶기
question_user_map = df.groupby('question_piece_id')['user_id'].apply(list)

# 결과 확인
print(question_user_map)


KeyError: 'question_piece_id'

In [18]:
from itertools import combinations
from collections import defaultdict

# 후보 쌍을 저장할 딕셔너리
pair_dict = defaultdict(set)

# 질문별로 고유한 후보 유저 ID 리스트 만들기
question_user_map_unique = df.groupby('question_piece_id')['user_id'].apply(lambda x: list(set(x)))


# 질문별 후보 리스트에서 쌍 저장
for qid, user_list in question_user_map_unique.items():
    for u1, u2 in combinations(user_list, 2):
        pair_dict[u1].add(u2)
        pair_dict[u2].add(u1)

# 상호 후보 관계 확인
mutual_pairs = []
for u1 in pair_dict:
    for u2 in pair_dict[u1]:
        if u1 in pair_dict[u2] and u1 < u2:  # 중복 제거용 정렬
            mutual_pairs.append((u1, u2))

print(f'서로 후보로 등장한 유저 쌍 개수: {len(mutual_pairs)}')
print(mutual_pairs[:10])


서로 후보로 등장한 유저 쌍 개수: 1082708
[(849444, 849922), (849444, 849928), (849444, 858120), (849444, 849938), (849444, 849946), (849444, 878628), (849444, 849963), (849444, 849974), (849444, 866359), (849444, 860215)]


In [19]:
unique_users = df['user_id'].nunique()
print(f'총 고유 유저 수: {unique_users}')

from math import comb
total_possible_pairs = comb(unique_users, 2)
print(f'전체 가능한 유저 쌍 수: {total_possible_pairs}')

mutual_pair_count = len(mutual_pairs)
print(f'상호 후보로 등장한 쌍 수: {mutual_pair_count}')

ratio = mutual_pair_count / total_possible_pairs
print(f'상호 후보 등장 비율: {ratio:.6f}')


총 고유 유저 수: 19994
전체 가능한 유저 쌍 수: 199870021
상호 후보로 등장한 쌍 수: 1082708
상호 후보 등장 비율: 0.005417


In [20]:
df. groupby('question_piece_id')['user_id'].apply(list)

question_piece_id
998458           [849444, 849454, 849460, 849469]
998459           [849446, 849466, 849497, 849498]
998460           [838642, 849445, 849454, 849620]
998461           [847375, 849452, 849475, 849558]
998462           [849445, 849477, 849502, 849620]
                             ...                 
208345628    [1006085, 1207183, 1221293, 1231232]
208345629    [1005921, 1207183, 1213853, 1228800]
208345630    [1208878, 1212910, 1238377, 1323127]
208351463    [1078712, 1430270, 1450729, 1552803]
208351468    [1011262, 1110794, 1430554, 1544061]
Name: user_id, Length: 1217589, dtype: object

In [21]:
# 모든 question별로 고유한 유저 리스트 만들기
question_user_map_unique = df.groupby('question_piece_id')['user_id'].apply(lambda x: list(set(x)))

# 유저쌍 등장 횟수 저장
from collections import Counter
from itertools import combinations

pair_counter = Counter()

for user_list in question_user_map_unique:
    for u1, u2 in combinations(sorted(user_list), 2):
        pair_counter[(u1, u2)] += 1

# 반대로 (u2, u1)도 카운팅해서 상호 등장 확인
mutual_pairs = [pair for pair, count in pair_counter.items() if pair_counter[pair] > 1]

# 또는 원래 쌍만 갖고 mutual set 다시 만들기
mutual_users = set()
for u1, u2 in mutual_pairs:
    mutual_users.update([u1, u2])



In [22]:
all_users = set(df['user_id'].unique())  # 총 19,994명

mutual_users = set()

for u1, u2 in mutual_pairs:
    mutual_users.update([u1, u2])

print(f'상호 지목 관계에 참여한 유저 수: {len(mutual_users)}')

non_mutual_users = all_users - mutual_users
print(f'상호 지목에 참여하지 않은 유저 수: {len(non_mutual_users)}')

상호 지목 관계에 참여한 유저 수: 16854
상호 지목에 참여하지 않은 유저 수: 3140


In [23]:
# 상호작용이 없는 유저 리스트
non_mutual_users = all_users - mutual_users

# 이 유저들이 등장한 질문 목록
non_mutual_df = df[df['user_id'].isin(non_mutual_users)]

# 각 question에서 등장한 후보 수
candidate_per_question = df.groupby('question_piece_id')['user_id'].nunique()

# 비상호 유저가 포함된 질문의 후보 수 평균
non_mutual_df['candidate_count'] = non_mutual_df['question_piece_id'].map(candidate_per_question)
print(non_mutual_df['candidate_count'].describe())


count    6905.000000
mean        3.952209
std         0.216708
min         2.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         4.000000
Name: candidate_count, dtype: float64


/tmp/ipykernel_18537/2336628669.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_mutual_df['candidate_count'] = non_mutual_df['question_piece_id'].map(candidate_per_question)


In [24]:
mutual_df = df[df['user_id'].isin(mutual_users)]
mutual_df['candidate_count'] = mutual_df['question_piece_id'].map(candidate_per_question)
print(mutual_df['candidate_count'].describe())


count    4.762704e+06
mean     3.937582e+00
std      2.474491e-01
min      1.000000e+00
25%      4.000000e+00
50%      4.000000e+00
75%      4.000000e+00
max      8.000000e+00
Name: candidate_count, dtype: float64


/tmp/ipykernel_18537/572803383.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mutual_df['candidate_count'] = mutual_df['question_piece_id'].map(candidate_per_question)


In [26]:
import os
import pandas as pd
from google.cloud import storage

# GCS에서 Parquet 파일 불러오기 및 형식 확인
print("\n--- 2. GCS Parquet 파일 로드 및 형식 확인 ---")
gcs_parquet_path = "gs://sprintda05_final_project/votes/polls_question.parquet"

print(f"📂 다음 Parquet 파일을 불러옵니다: {gcs_parquet_path}")

# pandas를 사용하여 Parquet 파일 불러오기
# 'engine'을 'pyarrow'로 명시하여 확실하게 pyarrow 사용
# gcsfs가 설치되어 있다면 pandas는 gs:// 경로를 자동으로 처리
df = pd.read_parquet(gcs_parquet_path, engine='pyarrow')

print("\n✅ Parquet 파일 로드 성공!")

print("\n--- 데이터프레임 상위 5개 행 (df.head()) ---")
print(df.head())

print("\n--- 데이터프레임 정보 (df.info()) ---")
# 각 컬럼의 이름, Non-Null 개수, 데이터 타입(Dtype) 확인
df.info()

print("\n--- 데이터프레임 기술 통계 (df.describe()) ---")
# 숫자형 컬럼에 대한 통계 정보 확인
print(df.describe(include='all')) # 모든 컬럼 타입 포함

print(f"\n--- 데이터프레임 행과 열 개수 (df.shape): {df.shape} ---")

print("\n--- 데이터프레임 컬럼 목록 (df.columns) ---")
print(df.columns.tolist())


--- 2. GCS Parquet 파일 로드 및 형식 확인 ---
📂 다음 Parquet 파일을 불러옵니다: gs://sprintda05_final_project/votes/polls_question.parquet

✅ Parquet 파일 로드 성공!

--- 데이터프레임 상위 5개 행 (df.head()) ---
    id                 question_text          created_at
0   99            가장 신비한 매력이 있는 사람은? 2023-03-31 15:22:53
1  100  "이 사람으로 한 번 살아보고 싶다" 하는 사람은? 2023-03-31 15:22:53
2  101                     미래의 틱톡커는? 2023-03-31 15:22:54
3  102               여기서 제일 특이한 친구는? 2023-03-31 15:22:54
4  103               가장 지켜주고 싶은 사람은? 2023-03-31 15:22:55

--- 데이터프레임 정보 (df.info()) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5025 entries, 0 to 5024
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             5025 non-null   int64         
 1   question_text  5025 non-null   object        
 2   created_at     5025 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 117.9+ KB

--- 데이터프레임 기술 통계

In [27]:
df

,id,question_text,created_at
0,99,가장 신비한 매력이 있는 사람은?,2023-03-31 15:22:53
1,100,"""이 사람으로 한 번 살아보고 싶다"" 하는 사람은?",2023-03-31 15:22:53
2,101,미래의 틱톡커는?,2023-03-31 15:22:54
3,102,여기서 제일 특이한 친구는?,2023-03-31 15:22:54
4,103,가장 지켜주고 싶은 사람은?,2023-03-31 15:22:55
...,...,...,...
5020,5129,나에게 가장 중요한 사람은?,2023-06-06 06:15:52
5021,5130,오목을 제일 잘 할 것 같은 사람은?,2023-06-06 06:15:52
5022,5131,가방에서 쓰레기가 안 나올 것 같은 사람은?,2023-06-06 06:15:52
5023,5132,아무리 많은 숙제도 30분만에 다 끝내버릴 수 있을 것 같은 친구는?,2023-06-06 06:15:52


In [30]:
df['question_text'].value_counts()

question_text
vote                    56
인생 2회차인 것 같은 사람은?        3
2세가 가장 귀여울 것 같은 사람은?     3
눈이 제일 큰 사람은?             3
지금 뭐하는지 궁금한 친구           3
                        ..
장기자랑 1등 할 것 같은 친구는?      1
키가 제일 클 것 같은 사람은?        1
한번쯤 예쁘다고 생각해본 사람은?       1
인스타 부계가 가장 맛깔나는 친구는?     1
개학 하자마자 인사 될것 같은 사람      1
Name: count, Length: 3903, dtype: int64

In [31]:
filtered_df = df[df['question_text'].str.contains('vote', na=False)]


In [33]:
filtered_df.head(5)

,id,question_text,created_at
87,186,vote,2023-04-01 11:09:15
384,483,vote,2023-05-02 05:33:11
540,639,vote,2023-05-11 15:52:44
587,696,vote,2023-05-15 13:58:24
603,712,vote,2023-05-15 13:58:30


In [53]:
import os
import pandas as pd
from google.cloud import storage

# GCS에서 Parquet 파일 불러오기 및 형식 확인
print("\n--- 2. GCS Parquet 파일 로드 및 형식 확인 ---")
gcs_parquet_path = "gs://sprintda05_final_project/votes/polls_questionpiece.parquet"

print(f"📂 다음 Parquet 파일을 불러옵니다: {gcs_parquet_path}")

# pandas를 사용하여 Parquet 파일 불러오기
# 'engine'을 'pyarrow'로 명시하여 확실하게 pyarrow 사용
# gcsfs가 설치되어 있다면 pandas는 gs:// 경로를 자동으로 처리
polls_questionpiece = pd.read_parquet(gcs_parquet_path, engine='pyarrow')

print("\n✅ Parquet 파일 로드 성공!")

print("\n--- 데이터프레임 상위 5개 행 (df.head()) ---")
print(polls_questionpiece.head())

print("\n--- 데이터프레임 정보 (df.info()) ---")
# 각 컬럼의 이름, Non-Null 개수, 데이터 타입(Dtype) 확인
polls_questionpiece.info()

print("\n--- 데이터프레임 기술 통계 (df.describe()) ---")
# 숫자형 컬럼에 대한 통계 정보 확인
print(polls_questionpiece.describe(include='all')) # 모든 컬럼 타입 포함

print(f"\n--- 데이터프레임 행과 열 개수 (df.shape): {polls_questionpiece.shape} ---")

print("\n--- 데이터프레임 컬럼 목록 (df.columns) ---")
print(polls_questionpiece.columns.tolist())


--- 2. GCS Parquet 파일 로드 및 형식 확인 ---
📂 다음 Parquet 파일을 불러옵니다: gs://sprintda05_final_project/votes/polls_questionpiece.parquet

✅ Parquet 파일 로드 성공!

--- 데이터프레임 상위 5개 행 (df.head()) ---
       id  is_voted          created_at  question_id  is_skipped
0  998458         1 2023-04-28 12:27:22          252           0
1  998459         1 2023-04-28 12:27:22          244           0
2  998460         1 2023-04-28 12:27:22          183           0
3  998461         1 2023-04-28 12:27:22          101           0
4  998462         1 2023-04-28 12:27:22          209           0

--- 데이터프레임 정보 (df.info()) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1265476 entries, 0 to 1265475
Data columns (total 5 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   id           1265476 non-null  int64         
 1   is_voted     1265476 non-null  int64         
 2   created_at   1265476 non-null  datetime64[ns]
 3   question_id  1265476 non

In [54]:
polls_questionpiece

,id,is_voted,created_at,question_id,is_skipped
0,998458,1,2023-04-28 12:27:22,252,0
1,998459,1,2023-04-28 12:27:22,244,0
2,998460,1,2023-04-28 12:27:22,183,0
3,998461,1,2023-04-28 12:27:22,101,0
4,998462,1,2023-04-28 12:27:22,209,0
...,...,...,...,...,...
1265471,208385226,0,2024-05-07 11:32:30,960,0
1265472,208385227,0,2024-05-07 11:32:30,1402,0
1265473,208385228,0,2024-05-07 11:32:30,1676,0
1265474,208385229,0,2024-05-07 11:32:30,3115,0


In [55]:
polls_questionpiece['question_id']

0           252
1           244
2           183
3           101
4           209
           ... 
1265471     960
1265472    1402
1265473    1676
1265474    3115
1265475    1461
Name: question_id, Length: 1265476, dtype: int64

In [56]:
import os
import pandas as pd
from google.cloud import storage

# GCS에서 Parquet 파일 불러오기 및 형식 확인
print("\n--- 2. GCS Parquet 파일 로드 및 형식 확인 ---")
gcs_parquet_path = "gs://sprintda05_final_project/votes/polls_questionset.parquet"

print(f"📂 다음 Parquet 파일을 불러옵니다: {gcs_parquet_path}")

# pandas를 사용하여 Parquet 파일 불러오기
# 'engine'을 'pyarrow'로 명시하여 확실하게 pyarrow 사용
# gcsfs가 설치되어 있다면 pandas는 gs:// 경로를 자동으로 처리
polls_questionset = pd.read_parquet(gcs_parquet_path, engine='pyarrow')

print("\n✅ Parquet 파일 로드 성공!")

print("\n--- 데이터프레임 상위 5개 행 (df.head()) ---")
print(polls_questionset.head())

print("\n--- 데이터프레임 정보 (df.info()) ---")
# 각 컬럼의 이름, Non-Null 개수, 데이터 타입(Dtype) 확인
polls_questionset.info()

print("\n--- 데이터프레임 기술 통계 (df.describe()) ---")
# 숫자형 컬럼에 대한 통계 정보 확인
print(polls_questionset.describe(include='all')) # 모든 컬럼 타입 포함

print(f"\n--- 데이터프레임 행과 열 개수 (df.shape): {polls_questionset.shape} ---")

print("\n--- 데이터프레임 컬럼 목록 (df.columns) ---")
print(polls_questionset.columns.tolist())


--- 2. GCS Parquet 파일 로드 및 형식 확인 ---
📂 다음 Parquet 파일을 불러옵니다: gs://sprintda05_final_project/votes/polls_questionset.parquet

✅ Parquet 파일 로드 성공!

--- 데이터프레임 상위 5개 행 (df.head()) ---
      id                             question_piece_id_list  \
0  99817  [998458, 998459, 998460, 998461, 998462, 99846...   
1  99830  [998588, 998589, 998590, 998591, 998592, 99859...   
2  99840  [998689, 998691, 998693, 998695, 998697, 99869...   
3  99841  [998688, 998690, 998692, 998694, 998696, 99869...   
4  99848  [998768, 998769, 998770, 998771, 998772, 99877...   

         opening_time status          created_at  user_id  
0 2023-04-28 12:27:22      F 2023-04-28 12:27:23   849436  
1 2023-04-28 12:28:07      F 2023-04-28 12:28:07   849438  
2 2023-04-28 12:28:38      F 2023-04-28 12:28:38   847375  
3 2023-04-28 12:28:38      F 2023-04-28 12:28:38   849446  
4 2023-04-28 12:28:57      F 2023-04-28 12:28:57   849477  

--- 데이터프레임 정보 (df.info()) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex:

In [57]:
polls_questionset

,id,question_piece_id_list,opening_time,status,created_at,user_id
0,99817,"[998458, 998459, 998460, 998461, 998462, 99846...",2023-04-28 12:27:22,F,2023-04-28 12:27:23,849436
1,99830,"[998588, 998589, 998590, 998591, 998592, 99859...",2023-04-28 12:28:07,F,2023-04-28 12:28:07,849438
2,99840,"[998689, 998691, 998693, 998695, 998697, 99869...",2023-04-28 12:28:38,F,2023-04-28 12:28:38,847375
3,99841,"[998688, 998690, 998692, 998694, 998696, 99869...",2023-04-28 12:28:38,F,2023-04-28 12:28:38,849446
4,99848,"[998768, 998769, 998770, 998771, 998772, 99877...",2023-04-28 12:28:57,F,2023-04-28 12:28:57,849477
...,...,...,...,...,...,...
158379,20838253,"[208383291, 208383292, 208383293, 208383294, 2...",2024-05-05 14:46:27,C,2024-05-05 14:06:27,1251933
158380,20838344,"[208384201, 208384202, 208384203, 208384204, 2...",2024-05-06 11:38:20,C,2024-05-06 10:58:20,876072
158381,20838419,"[208384951, 208384952, 208384953, 208384954, 2...",2024-05-07 00:55:00,C,2024-05-07 00:15:00,1208878
158382,20838445,"[208385211, 208385212, 208385213, 208385214, 2...",2024-05-07 12:09:08,C,2024-05-07 11:29:08,1001607


In [58]:
polls_questionset['question_piece_id_list']

0         [998458, 998459, 998460, 998461, 998462, 99846...
1         [998588, 998589, 998590, 998591, 998592, 99859...
2         [998689, 998691, 998693, 998695, 998697, 99869...
3         [998688, 998690, 998692, 998694, 998696, 99869...
4         [998768, 998769, 998770, 998771, 998772, 99877...
                                ...                        
158379    [208383291, 208383292, 208383293, 208383294, 2...
158380    [208384201, 208384202, 208384203, 208384204, 2...
158381    [208384951, 208384952, 208384953, 208384954, 2...
158382    [208385211, 208385212, 208385213, 208385214, 2...
158383    [208385221, 208385222, 208385223, 208385224, 2...
Name: question_piece_id_list, Length: 158384, dtype: object

In [59]:
print(polls_questionset.loc[1, 'question_piece_id_list']),
print(polls_questionset.loc[30, 'question_piece_id_list']),
print(polls_questionset.loc[150, 'question_piece_id_list']),
print(polls_questionset.loc[50392, 'question_piece_id_list']),
print(polls_questionset.loc[92845, 'question_piece_id_list']),
print(polls_questionset.loc[100334, 'question_piece_id_list']),
print(polls_questionset.loc[158382, 'question_piece_id_list'])

[998588, 998589, 998590, 998591, 998592, 998593, 998594, 998595, 998596, 998597]
[1000018, 1000019, 1000020, 1000021, 1000022, 1000023, 1000024, 1000025, 1000026, 1000027]
[1015598, 1015599, 1015600, 1015601, 1015602, 1015603, 1015604, 1015605, 1015606, 1015607]
[35791718, 35791719, 35791720, 35791721, 35791722, 35791723, 35791724, 35791725, 35791726, 35791727]
[84290303, 84290307, 84290311, 84290314, 84290319, 84290323, 84290325, 84290328, 84290332, 84290337]
[94394169, 94394172, 94394175, 94394178, 94394182, 94394186, 94394192, 94394197, 94394201, 94394206]
[208385211, 208385212, 208385213, 208385214, 208385215, 208385216, 208385217, 208385218, 208385219, 208385220]


In [60]:
type(polls_questionset.loc[0, 'question_piece_id_list'])

str

In [61]:
import ast  # 안전한 문자열 → 리스트 변환 도구

# 문자열 → 리스트로 변환 후 길이 측정
polls_questionset['question_count'] = polls_questionset['question_piece_id_list'].apply(lambda x: len(ast.literal_eval(x)))


In [64]:
polls_questionset_exploded = polls_questionset.explode('question_piece_id_list').reset_index(drop=True)


In [65]:
polls_questionset_exploded

,id,question_piece_id_list,opening_time,status,created_at,user_id,question_count
0,99817,"[998458, 998459, 998460, 998461, 998462, 99846...",2023-04-28 12:27:22,F,2023-04-28 12:27:23,849436,10
1,99830,"[998588, 998589, 998590, 998591, 998592, 99859...",2023-04-28 12:28:07,F,2023-04-28 12:28:07,849438,10
2,99840,"[998689, 998691, 998693, 998695, 998697, 99869...",2023-04-28 12:28:38,F,2023-04-28 12:28:38,847375,10
3,99841,"[998688, 998690, 998692, 998694, 998696, 99869...",2023-04-28 12:28:38,F,2023-04-28 12:28:38,849446,10
4,99848,"[998768, 998769, 998770, 998771, 998772, 99877...",2023-04-28 12:28:57,F,2023-04-28 12:28:57,849477,10
...,...,...,...,...,...,...,...
158379,20838253,"[208383291, 208383292, 208383293, 208383294, 2...",2024-05-05 14:46:27,C,2024-05-05 14:06:27,1251933,10
158380,20838344,"[208384201, 208384202, 208384203, 208384204, 2...",2024-05-06 11:38:20,C,2024-05-06 10:58:20,876072,10
158381,20838419,"[208384951, 208384952, 208384953, 208384954, 2...",2024-05-07 00:55:00,C,2024-05-07 00:15:00,1208878,10
158382,20838445,"[208385211, 208385212, 208385213, 208385214, 2...",2024-05-07 12:09:08,C,2024-05-07 11:29:08,1001607,10


In [ ]:
merged = polls_questionset.merge(question_piece_id_list, left_on='question_piece_id_list', right_on='id', how='left')
